# CTC architectures

CTC veya Connectionist Temporal Classification, otomatik konuşma tanıma için **yalnızca kodlayıcı** dönüştürücü modelleri ile kullanılan bir tekniktir. Bu tür modellere örnek olarak **Wav2Vec2**, **HuBERT** ve **M-CTC-T** verilebilir.

**Yalnızca kodlayıcı dönüştürücü** en basit dönüştürücü türüdür çünkü modelin yalnızca kodlayıcı kısmını kullanır. Kodlayıcı, giriş dizisini (ses dalga formu) okur ve bunu, **çıkış gömülmeleri olarak da bilinen bir dizi gizli duruma eşler**.

Bir CTC modeliyle, sınıf etiketi tahminleri elde etmek için gizli durumlar dizisine ek bir doğrusal eşleme uygularız. **Sınıf etiketleri alfabenin karakterleridir** (a, b, c, ...). Bu şekilde, hedef dildeki herhangi bir kelimeyi küçük bir sınıflandırma kafasıyla tahmin edebiliyoruz, çünkü kelime dağarcığının sadece 26 karakter artı birkaç özel belirteçten oluşması gerekiyor.

![](https://huggingface.co/datasets/huggingface-course/audio-course-images/resolve/main/wav2vec2-ctc.png)

Şimdiye kadar bu, NLP'de BERT gibi bir modelle yaptığımıza çok benziyor: yalnızca kodlayıcı transformer modeli, metin jetonlarımızı bir dizi kodlayıcı gizli durumuna eşler ve ardından her gizli durum için bir sınıf etiketi tahmini elde etmek için doğrusal bir eşleme uygularız.

Sorun şu: Konuşmada, ses girişlerinin ve metin çıkışlarının hizalanmasını bilmiyoruz. Konuşmanın söylendiği sıranın metnin yazıya döküldüğü sırayla aynı olduğunu biliyoruz (hizalama monotonik olarak adlandırılır), **ancak yazıya dökülen karakterlerin sesle nasıl hizalandığını bilmiyoruz**. İşte CTC algoritması burada devreye giriyor.

## Alignment

Otomatik konuşma tanıma veya ASR, sesi girdi olarak alıp çıktı olarak metin üretmeyi içerir. Metni nasıl tahmin edeceğimiz konusunda birkaç seçeneğimiz var:

- tek tek karakterler olarak
- fonemler olarak
- kelime belirteçleri olarak

Bir ASR modeli, metnin ses dosyasının insan yapımı bir transkripsiyonu olduğu (ses, metin) çiftlerinden oluşan bir veri kümesi üzerinde eğitilir. Genellikle veri kümesi, ses dosyasında hangi kelimenin veya hecenin nerede geçtiğini belirten herhangi bir zamanlama bilgisi içermez. Eğitim sırasında zamanlama bilgisine güvenemeyeceğimiz için, giriş ve çıkış dizilerinin nasıl hizalanması gerektiği konusunda hiçbir fikrimiz yoktur.

Girdimizin bir saniyelik bir ses dosyası olduğunu varsayalım. Wav2Vec2'de model ilk olarak CNN özellik kodlayıcısını kullanarak ses girişini daha kısa bir gizli durum dizisine indirger; **burada her 20 milisaniye ses için bir gizli durum vektörü vardır**. Daha sonra bir saniyelik ses için 50 gizli durumdan oluşan bir diziyi dönüştürücü kodlayıcıya iletiyoruz. (Giriş dizisinden çıkarılan ses segmentleri **kısmen üst üste gelir**, bu nedenle her 20 ms'de bir gizli durum vektörü yayılsa da, her gizli durum aslında 25 ms'lik sesi temsil eder).

Dönüştürücü kodlayıcı, bu gizli durumların her biri için bir özellik temsili öngörür, yani dönüştürücüden 50 çıktı dizisi alırız. Bu çıktıların her biri 768 boyutluluğa sahiptir. Dolayısıyla bu örnekte transformatör kodlayıcının çıktı dizisi (768, 50) şeklindedir. Bu tahminlerin her biri, bir fonem süresinden daha kısa olan 25 ms'lik bir süreyi kapsadığından, tek tek fonemleri veya karakterleri tahmin etmek mantıklıdır, ancak tüm kelimeleri değil. CTC en iyi küçük bir kelime dağarcığı ile çalışır, **bu yüzden karakterleri tahmin edeceğiz**.

![](https://huggingface.co/datasets/huggingface-course/audio-course-images/resolve/main/cnn-feature-encoder.png)

Metin tahminleri yapmak için 768 boyutlu kodlayıcı çıktılarının her birini doğrusal bir katman ("CTC kafası") kullanarak karakter etiketlerimizle eşleştiriyoruz. Model daha sonra logitleri içeren bir (50, 32) tensör tahmin eder, burada 32 kelime haznesindeki belirteç sayısıdır. Sekanstaki özelliklerin her biri için bir tahmin yaptığımızdan, sesin her saniyesi için toplam 50 karakter tahmini elde ederiz.

Bununla birlikte, her 20 ms'de bir karakter tahmin edersek, çıktı dizimiz aşağıdaki gibi görünebilir:

`BRIIONSAWWSOMEETHINGCLOSETOPANICONHHISOPPONENT'SSFAACEWHENTHEMANNFINALLLYRREECOGGNNIIZEDHHISSERRRRORR ...`

Yakından bakarsanız, bir şekilde İngilizceye benziyor **ancak karakterlerin çoğu çoğaltılmış**. Bunun nedeni, modelin giriş dizisindeki **her 20 ms'lik ses için bir çıktı** vermesi gerektiğinden ve **bir karakter 20 ms'den daha uzun bir süreye yayılırsa, çıktıda birden çok kez görünecektir**. Bundan kaçınmanın bir yolu yoktur, özellikle de eğitim sırasında transkriptin zamanlamasının ne olduğunu bilmediğimiz için. **CTC bu kopyaları filtrelemenin bir yoludur.**

(Gerçekte, tahmin edilen dizi, model sesin neyi temsil ettiğinden tam olarak emin olmadığında veya karakterler arasındaki boş alan için çok sayıda dolgu belirteci de içerir. Anlaşılır olması için bu dolgu belirteçlerini örnekten çıkardık. Ses segmentleri arasındaki kısmi örtüşme, karakterlerin çıktıda yinelenmesinin bir başka nedenidir).

## The CTC algorithm

CTC algoritmasının anahtarı, **genellikle boş belirteç olarak adlandırılan özel bir belirteç kullanmaktır**. Bu, **modelin tahmin edeceği başka bir belirteçtir ve kelime dağarcığının bir parçasıdır**. Bu örnekte, boş belirteç _ olarak gösterilmiştir. Bu özel belirteç, karakter grupları arasında sert bir sınır görevi görür.

CTC modelinden elde edilen tam çıktı aşağıdaki gibi olabilir:

`B_R_II_O_N_||_S_AWW_|||||_S_OMEE_TH_ING_||_C_L_O_S_E||TO|_P_A_N_I_C_||_ON||HHI_S||_OP_P_O_N_EN_T_'SS||_F_AA_C_E||_W_H_EN||THE||M_A_NN_||||_F_I_N_AL_LL_Y||||_RREE_C_O_GG_NN_II_Z_ED|||HHISS|||_ER_RRR_ORR||||`

| belirteci kelime ayırıcı karakterdir. Örnekte boşluk yerine | kullandık, böylece kelime sonlarının nerede olduğunu tespit etmek kolaylaştı, ancak aynı amaca hizmet ediyor.

CTC boşluk karakteri, yinelenen karakterleri filtrelemeyi mümkün kılar. Örneğin, tahmin edilen dizideki son kelimeye bakalım, _ER_RR_ORR. CTC boşluk belirteci olmadan kelime şöyle görünürdü:

`ERRRRORR`

Eğer sadece yinelenen karakterleri kaldırırsak, bu EROR olur. Bunun doğru yazım olmadığı açıktır. **Ancak CTC boş belirteci ile her gruptaki kopyaları kaldırabiliriz**, böylece:

`_ER_RRR_ORR` -> `_ER_R_OR`

ve şimdi son kelimeyi elde etmek için _ boş belirtecini kaldırıyoruz:

`ERROR`

Bu mantığı | dahil olmak üzere tüm metne uygularsak ve kalan | karakterlerini boşlukla değiştirirsek, nihai CTC deşifre çıktısı şu şekilde olur:

`BRION SAW SOMETHING CLOSE TO PANIC ON HIS OPPONENT'S FACE WHEN THE MAN FINALLY RECOGNIZED HIS ERROR`

Özetlemek gerekirse, **model giriş dalga formundaki her 20 ms'lik (kısmen örtüşen) ses için bir belirteç (karakter) tahmin eder**. Bu da çok sayıda kopya oluşturur. CTC boş belirteci sayesinde, kelimelerin doğru yazımını bozmadan bu kopyaları kolayca kaldırabiliriz. Bu, çıktı metnini giriş sesiyle hizalama sorununu çözmenin çok basit ve kullanışlı bir yoludur.

Bir transformatör kodlayıcı modeline CTC eklemek kolaydır: kodlayıcıdan gelen çıkış dizisi, akustik özellikleri kelime dağarcığına yansıtan doğrusal bir katmana gider. Model özel bir CTC kaybı ile eğitilir.

**CTC'nin bir dezavantajı, kulağa doğru gelen ancak doğru yazılmayan kelimeler üretebilmesidir**. Sonuçta, CTC kafası yalnızca tek tek karakterleri dikkate alır, tam kelimeleri değil. **Ses transkripsiyonlarının kalitesini artırmanın bir yolu da harici bir dil modeli kullanmaktır**. Bu dil modeli esasen CTC çıktısının üstünde bir yazım denetimi görevi görür.

## What’s the difference between Wav2Vec2, HuBERT, M-CTC-T, …?

Tüm transformatör tabanlı CTC modelleri çok benzer bir mimariye sahiptir: transformatör kodlayıcısını (ancak kod çözücüyü değil) üstte bir CTC kafası ile kullanırlar. Mimari açıdan farklı olmaktan çok benzerdirler.

Wav2Vec2 ve M-CTC-T arasındaki bir fark, **birincisinin ham ses dalga formları üzerinde çalışırken ikincisinin girdi olarak mel spektrogramları kullanmasıdır**. Modeller ayrıca farklı amaçlar için eğitilmiştir. Örneğin M-CTC-T, çok dilli konuşma tanıma için eğitilmiştir ve bu nedenle diğer alfabelere ek olarak Çince karakterleri de içeren nispeten büyük bir CTC kafasına sahiptir.

Wav2Vec2 & HuBERT **tamamen aynı mimariyi kullanır** ancak çok farklı şekillerde eğitilir. **Wav2Vec2, BERT'in maskeli dil modellemesi gibi, sesin maskeli kısımları için konuşma birimlerini tahmin ederek önceden eğitilmiştir**. HuBERT, BERT'ten aldığı ilhamı bir adım öteye taşıyor ve bir metin cümlesindeki belirteçlere benzeyen "ayrık konuşma birimlerini" tahmin etmeyi öğreniyor, böylece konuşma yerleşik NLP teknikleri kullanılarak ele alınabiliyor.

Açıklığa kavuşturmak gerekirse, burada vurgulanan modeller tek transformatör tabanlı CTC modelleri değildir. Başka modeller de var, ancak artık hepsinin benzer şekilde çalıştığını biliyorsunuz.